## Imports

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing.image import DirectoryIterator
from keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from tensorflow import keras
from keras.models import Model
import os
from pathlib import Path
import io
from tqdm import tqdm
import cv2
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
from numpy import sqrt
from numpy import argmax
import seaborn as sns
import math
import sklearn
from sklearn.metrics import confusion_matrix, multilabel_confusion_matrix, roc_curve
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
import pandas as pd

## Classes to handle dataset images

In [ ]:
class DatasetHandler:
    def __init__(self,
                 model_path,
                 dataset,
                 img_width=600,
                 img_height=200,
                 batch_size=32):
        
        self.model_path = model_path
        self.dataset = dataset
        self.obstacle_dataset = os.path.join(dataset, 'obstacle')
        self.no_obstacle_dataset = os.path.join(dataset, 'no_obstacle')
        self.img_width = img_width
        self.img_height = img_height
        self.obstacle_images = []
        self.no_obstacle_images = []
        self.sdv_images = []
        self._update_image_lists = False
        self.batch_size = batch_size
            
        # Will be determined while reading all images from dataset
        self.num_obstacles = 0
        self.num_no_obstacles = 0
        self.num_sdvs = 0
        self.num_images = 0
            
        (self.obstacle_image_names,
        self.num_obstacles) = self._get_all_dataset_image_names(self.dataset, 'obstacle')
        (self.no_obstacle_image_names,
        self.num_no_obstacles) = self._get_all_dataset_image_names(self.dataset, 'no_obstacle')
        self.datagen, self.steps = self.get_datagen(self.batch_size)
                            
    def _get_all_dataset_image_names(self, dataset, class_name):
        class_dataset = os.path.join(dataset, class_name)
        image_names = os.listdir(class_dataset)
        image_paths = [os.path.join(class_dataset, image_name) for image_name in image_names]
        
        return image_paths, len(image_paths)
            
    def _get_image(self, imname):
        imrgb = cv2.imread(imname)
        im = cv2.cvtColor(imrgb, cv2.COLOR_BGR2GRAY) 
        
        return im
    
    def get_datagen(self, batch_size):
        datagen = ImageDataGenerator(rescale=1./255)
        it = DirectoryIterator(self.dataset, image_data_generator=datagen, batch_size=batch_size)
        steps= it.__len__()
        
        return datagen, steps

    # Special generator to generate the 3 parts of the input image as 3 separate input images
    def three_im_generator(self, gen, dataset, target_size, batch_size, class_mode):

        im_gen = gen.flow_from_directory(dataset, 
                                         target_size=target_size, 
                                         batch_size=batch_size,
                                         shuffle=False,
                                         color_mode='grayscale',
                                         class_mode=class_mode)
        self.filenames = im_gen.filenames

        while True:
            im1_s, im2_s, im3_s = [], [], []
            images, labels = im_gen.next()

            for im in images:
                w = im.shape[1]
                im1 = im[:, :w//3]
                im2 = im[:, w//3:(w*2)//3] 
                im3 = im[:, (w*2)//3:] 
                im1_s.append(im1)
                im2_s.append(im2)
                im3_s.append(im3)

            im1_s = np.array(im1_s)
            im2_s = np.array(im2_s)
            im3_s = np.array(im3_s)
            yield [im1_s, im2_s, im3_s], labels
                        
    def get_metrics(self, model, color_mode='rgb'):
        eval_generator = self.three_im_generator(
            self.datagen,
            self.dataset,
            target_size=(self.img_height, self.img_width),
            batch_size=self.batch_size,
            class_mode='binary'
        )
        
        return model.evaluate(eval_generator, steps=self.steps, verbose=1)
        
    def get_predictions(self, model, color_mode='rgb'):
        predict_generator = self.three_im_generator(
            self.datagen,
            self.dataset,
            target_size=(self.img_height, self.img_width),
            batch_size=self.batch_size,
            class_mode='binary'
        )
        
        return model.predict(predict_generator, steps=self.steps, verbose=1)
        
    def print_model_metrics(self, model, color_mode):
        metrics = self.get_metrics(model, color_mode)

        for name, value in zip(model.metrics_names, metrics):
            print(name, ': ', value)
            
    # Compute and plot multi-class confusion-matrix
    def plot_cm(self, model_path, labels, predictions, threshold):
        cm = confusion_matrix(labels, predictions > threshold) 
        print(f'cm: {cm}')
        sns.heatmap(cm, annot=True, fmt='d')
        plt.title('Confusion matrix')
        plt.ylabel('Actual label')
        plt.xlabel('Predicted label') 

    # Compute and plot multi-class confusion-matrix with normalization
    def plot_cm_normalized(self, model_path, labels, predictions, threshold):
        cm = confusion_matrix(labels, predictions > threshold)
        # Normalise
        cmn = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        fig, ax = plt.subplots(figsize=(5, 5))
        sns.heatmap(cmn, annot=True, fmt='.3f')
        plt.ylabel('Actual')
        plt.xlabel('Predicted')
        plt.show(block=False)  
                
    def display_false_positives(self, predictions, threshold):
        image_names = [s.split('/')[-1] for s in self.no_obstacle_image_names]
        false_positives = [fname for i, fname in enumerate(self.filenames)\
                           if fname.split('/')[-1] in image_names\
                           and predictions[i] > threshold]
        preds = [p for i, p in enumerate(predictions)\
                 if self.filenames[i].split('/')[-1] in image_names\
                 and p > threshold]

        if 500 > len(false_positives) > 1:
            num_images = len(false_positives)
            _, axarr = plt.subplots(num_images, 1, figsize=(num_images, num_images))

            for i, fname in enumerate(false_positives):
                imname = fname.split('/')[-1]
                impath = os.path.join(self.no_obstacle_dataset, imname)
                im = cv2.imread(impath)
                print(f'FP prediction: {preds[i]}, imname: {imname}')
                im = cv2.cvtColor(im, cv2.COLOR_BGR2RGB)
                axarr[i].imshow(im)
                axarr[i].set_title(str(preds[i]) + ' ' + imname + (' '*30))
            plt.show()
        elif false_positives:
            fname = false_positives[0]
            imname = fname.split('/')[-1]
            impath = os.path.join(self.no_obstacle_dataset, imname)
            im = cv2.imread(impath)
            plt.imshow(im)
            plt.title(str(preds[0]) + ' ' + imname)
            plt.show()
            
    def display_false_negatives(self, predictions, threshold):
        image_names = [s.split('/')[-1] for s in self.obstacle_image_names]
        false_negatives = [fname for i, fname in enumerate(self.filenames)\
                           if fname.split('/')[-1] in image_names\
                           and predictions[i] <= threshold]
        preds = [p for i, p in enumerate(predictions)\
                 if self.filenames[i].split('/')[-1] in image_names\
                 and p <= threshold]

        if 500 > len(false_negatives) > 1:
            num_images = len(false_negatives)
            _, axarr = plt.subplots(num_images, 1, figsize=(num_images, num_images))

            for i, fname in enumerate(false_negatives):
                imname = fname.split('/')[-1]
                impath = os.path.join(self.obstacle_dataset, imname)
                im = cv2.imread(impath)
                im = cv2.cvtColor(im, cv2.COLOR_BGR2RGB)
                axarr[i].imshow(im)
                axarr[i].set_title(str(preds[i]) + ' ' + imname + (' '*30))
            plt.show()
        elif false_negatives:
            fname = false_positives[0]
            imname = fname.split('/')[-1]
            impath = os.path.join(self.obstacle_dataset, imname)
            im = cv2.imread(impath)
            plt.imshow(im)
            plt.title(str(preds[0]) + ' ' + imname)
            plt.show()   
            
    def save_false_positives(self, predictions, threshold, save_path):
        image_names = [s.split('/')[-1] for s in self.no_obstacle_image_names]
        false_positives = [fname for i, fname in enumerate(self.filenames)\
                           if fname.split('/')[-1] in image_names\
                           and predictions[i] > threshold]
        preds = [p for i, p in enumerate(predictions)\
                 if self.filenames[i].split('/')[-1] in image_names\
                 and p > threshold]

        fp_path = os.path.join(save_path, 'false_positives')
        Path(fp_path).mkdir(parents=True, exist_ok=True)
        for i, fname in enumerate(false_positives):
            imname = fname.split('/')[-1]
            in_path = os.path.join(self.no_obstacle_dataset, imname)
            im = cv2.imread(in_path)
            out_path = os.path.join(fp_path, imname)
            cv2.imwrite(out_path, im)
            
    def save_true_positives(self, predictions, threshold, save_path):
        image_names = [s.split('/')[-1] for s in self.obstacle_image_names]
        true_positives = [fname for i, fname in enumerate(self.filenames)\
                           if fname.split('/')[-1] in image_names\
                           and predictions[i] > threshold]
        preds = [p for i, p in enumerate(predictions)\
                 if self.filenames[i].split('/')[-1] in image_names\
                 and p > threshold]

        fp_path = os.path.join(save_path, 'true_positives')
        Path(fp_path).mkdir(parents=True, exist_ok=True)
        for i, fname in enumerate(true_positives):
            imname = fname.split('/')[-1]
            in_path = os.path.join(self.obstacle_dataset, imname)
            im = cv2.imread(in_path)
            out_path = os.path.join(fp_path, imname)
            cv2.imwrite(out_path, im)

    def save_false_negatives(self, predictions, threshold, save_path):
        image_names = [s.split('/')[-1] for s in self.obstacle_image_names]
        false_negatives = [fname for i, fname in enumerate(self.filenames)\
                           if fname.split('/')[-1] in image_names\
                           and predictions[i] <= threshold]
        preds = [p for i, p in enumerate(predictions)\
                 if self.filenames[i].split('/')[-1] in image_names\
                 and p <= threshold]
        
        fn_path = os.path.join(save_path, 'false_negatives')
        Path(fn_path).mkdir(parents=True, exist_ok=True)
        for i, fname in enumerate(false_negatives):
            imname = fname.split('/')[-1]
            in_path = os.path.join(self.obstacle_dataset, imname)
            im = cv2.imread(in_path)
            out_path = os.path.join(fn_path, imname)
            cv2.imwrite(out_path, im)
            
    def save_true_negatives(self, predictions, threshold, save_path):
        image_names = [s.split('/')[-1] for s in self.no_obstacle_image_names]
        true_negatives = [fname for i, fname in enumerate(self.filenames)\
                           if fname.split('/')[-1] in image_names\
                           and predictions[i] <= threshold]
        preds = [p for i, p in enumerate(predictions)\
                 if self.filenames[i].split('/')[-1] in image_names\
                 and p <= threshold]
        
        fn_path = os.path.join(save_path, 'true_negatives')
        Path(fn_path).mkdir(parents=True, exist_ok=True)
        for i, fname in enumerate(true_negatives):
            imname = fname.split('/')[-1]
            in_path = os.path.join(self.no_obstacle_dataset, imname)
            im = cv2.imread(in_path)
            out_path = os.path.join(fn_path, imname)
            cv2.imwrite(out_path, im)

## Main

In [ ]:
# -- Variables --
model_path = '/home/drevital/cs_video_processor/models/humid_jan22_ggm'
dataset = '/home/drevital/obstacles_classification_datasets/humid_jan22_ggm/eval'
model_name = model_path.split('/')[-1]
color_mode = 'rgb'
threshold = 0.3
batch_size = 32
#save_base_path = '/home/drevital/obstacles_classification_datasets/model_eval'
save_name = dataset.split('/')[-2] + '_' + str(threshold)
save_path = os.path.join(save_base_path, save_name)

# -- Run the Evaluation --
model = tf.keras.models.load_model(model_path)
handler = DatasetHandler(model_path, dataset, batch_size=batch_size)

# -- Define Labels
labels = np.array([0]*handler.num_no_obstacles\
                + [1]*handler.num_obstacles)

# -- Predict with the model
predictions = handler.get_predictions(model, color_mode=color_mode)

# -- Print confision-matrix
handler.plot_cm_normalized(model_path, labels, predictions, threshold=threshold)

# -- Save Images
handler.save_false_negatives(predictions, threshold=threshold, save_path=save_path)
handler.save_false_positives(predictions, threshold=threshold, save_path=save_path)